In [ ]:
!pip install datasets transformers huggingface_hub
!pip install accelerate -U
!apt-get install git-lfs

import torch
import pandas as pd
import transformers
from datasets import Dataset
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer


torch.cuda.is_available()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


True

In [ ]:
path = "/content/test.csv"
path1='/content/train.csv'
df1 = pd.read_csv(path)
df2 = pd.read_csv(path1)
df = pd.concat([df1, df2])
df = df.reset_index(drop=True)
df

,text,label
0,"Contrary to other reviews, I have zero complai...",1
1,Last summer I had an appointment to get new ti...,0
2,"Friendly staff, same starbucks fair you get an...",1
3,The food is good. Unfortunately the service is...,0
4,Even when we didn't have a car Filene's Baseme...,1
...,...,...
597995,Ryan was as good as everyone on yelp has claim...,1
597996,Professional \nFriendly\nOn time AND affordabl...,1
597997,Phone calls always go to voicemail and message...,0
597998,Looks like all of the good reviews have gone t...,0


In [ ]:
df.dropna(inplace = True)
df

,text,label
0,"Contrary to other reviews, I have zero complai...",1
1,Last summer I had an appointment to get new ti...,0
2,"Friendly staff, same starbucks fair you get an...",1
3,The food is good. Unfortunately the service is...,0
4,Even when we didn't have a car Filene's Baseme...,1
...,...,...
597995,Ryan was as good as everyone on yelp has claim...,1
597996,Professional \nFriendly\nOn time AND affordabl...,1
597997,Phone calls always go to voicemail and message...,0
597998,Looks like all of the good reviews have gone t...,0


In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [ ]:
def preprocess_text(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    review = soup.get_text()
    review = re.sub(r'\[[^]]*\]', ' ', review)
    review = re.sub(r'[^a-zA-Z]', ' ', review)
    review = review.split()
    return ' '.join(review)

In [ ]:
df['text'] = df['text'].apply(preprocess_text)

<ipython-input-5-b4d15d81e83b>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_text, "html.parser")
<ipython-input-5-b4d15d81e83b>:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(html_text, "html.parser")


In [ ]:
#df['sentiment'] = df['sentiment'].map({'positive':1,'negative':0})
len(df)

598000

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

## Assuming 'target' is the column you want to stratify based on
target_column = 'label'

# Create a StratifiedShuff Create a StratifiedShuffleSplit object for the initial split (10%)
initial_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.98, random_state=42)

# Get the indices for the initial stratified subset
initial_train_index, initial_test_index = next(initial_splitter.split(df, df[target_column]))

# Create the initial training and testing sets
initial_train_df, initial_test_df = df.iloc[initial_train_index], df.iloc[initial_test_index]

# Further split the initial training set into training and testing subsets (20%)
train_df, test_df = train_test_split(
    initial_train_df,
    test_size=0.2,
    stratify=initial_train_df[target_column],
    random_state=42
)
#len(final_test_df)

In [ ]:
#train_df, test_df = train_test_split(df, test_size=0.2)  # Adjust the test_size as needed

train_data_dict = {
    "text": train_df["text"].tolist(),
    "label": train_df["label"].tolist(),
}

test_data_dict = {
    "text": test_df["text"].tolist(),
    "label": test_df["label"].tolist(),
}

train_dataset = Dataset.from_dict(train_data_dict)
test_dataset = Dataset.from_dict(test_data_dict)

In [ ]:
model_name = "bert-base-uncased"  # You can choose a different BERT model as well
tokenizer = BertTokenizer.from_pretrained(model_name)
#model = BertForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(data):
    return tokenizer(data["text"], padding="max_length", truncation=True)




In [ ]:
# Define a custom model class for training from scratch
class BertForSequenceClassificationFromScratch(BertForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.init_weights()

# Initialize the custom model without loading pre-trained weights
num_labels = 2  # Binary classification
config = BertForSequenceClassification.from_pretrained(model_name).config
config.num_labels = num_labels
model = BertForSequenceClassificationFromScratch(config)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9568 [00:00<?, ? examples/s]

Map:   0%|          | 0/2392 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
from transformers import TrainingArguments, Trainer
from accelerate import Accelerator
from transformers import TrainerCallback
from copy import deepcopy

training_args = TrainingArguments(
    per_device_train_batch_size=8,
    output_dir="./sentiment_finetuned_model",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=2,
    num_train_epochs=10,
)

# Define an Accelerator to enable mixed-precision training and distributed training
accelerator = Accelerator()


In [ ]:
class CustomCallback(TrainerCallback):

    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer

    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy


In [ ]:
# Define a Trainer using the TrainingArguments and Accelerator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)
trainer.add_callback(CustomCallback(trainer))
# Train the model for 10 epochs and collect accuracy information
train_acc_list = []
eval_acc_list = []


trainresults=trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(trainresults)
print(eval_results)
# Collect accuracy information
#train_acc_list.append(eval_results["train_accuracy"])
#eval_acc_list.append(eval_results["eval_accuracy"])

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.720300,0.711991,0.500000,0.666667
2,0.707200,0.706148,0.500000,0.666667
3,0.706300,0.703967,0.500000,0.666667
4,0.704900,0.696356,0.500000,0.000000
5,0.703100,0.694526,0.500000,0.666667
6,0.700600,0.702389,0.500000,0.000000
7,0.699100,0.700272,0.500000,0.000000
8,0.699600,0.693249,0.500000,0.666667
9,0.699000,0.693399,0.500000,0.666667
10,0.698100,0.693152,0.500000,0.666667


<ipython-input-14-b571dbad15c2>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


TrainOutput(global_step=11960, training_loss=0.7038345387946802, metrics={'train_runtime': 12912.1954, 'train_samples_per_second': 7.41, 'train_steps_per_second': 0.926, 'total_flos': 2.51744657768448e+16, 'train_loss': 0.7038345387946802, 'epoch': 10.0})
{'eval_loss': 0.6931516528129578, 'eval_accuracy': 0.5, 'eval_f1': 0.6666666666666666, 'eval_runtime': 77.6954, 'eval_samples_per_second': 30.787, 'eval_steps_per_second': 3.848, 'epoch': 10.0}


In [ ]:

df=pd.DataFrame(trainer.state.log_history)


In [ ]:
df

,train_loss,train_accuracy,train_f1,train_runtime,train_samples_per_second,train_steps_per_second,epoch,step,loss,learning_rate,eval_loss,eval_accuracy,eval_f1,eval_runtime,eval_samples_per_second,eval_steps_per_second,total_flos
0,0.711991,0.5,0.666667,305.4361,31.326,3.916,1.0,1196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1196,0.7203,0.000045,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1196,NaN,NaN,0.711991,0.5,0.666667,77.8301,30.734,3.842,NaN
3,0.706148,0.5,0.666667,305.7872,31.290,3.911,2.0,2392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2392,0.7072,0.000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2392,NaN,NaN,0.706148,0.5,0.666667,77.8710,30.717,3.840,NaN
6,0.703967,0.5,0.666667,305.4357,31.326,3.916,3.0,3588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3588,0.7063,0.000035,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3588,NaN,NaN,0.703967,0.5,0.666667,78.2500,30.569,3.821,NaN
9,0.696356,0.5,0.000000,305.4849,31.321,3.915,4.0,4784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_excel("yelpscratch20.xlsx", index=False)

In [ ]:
from google.colab import files
files.download("yelpscratch20.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>